In [2]:
import pandas as pd
import numpy as np
import yaml
import os
import pandas as pd
import string
import random
import math

In [3]:
def read(type):
    type_file = f'./snli/snli_1.0/snli_1.0_{type}.jsonl'
    type_data = pd.read_json(type_file, lines=True)[['gold_label', 'sentence1', 'sentence2', 'pairID']]
    valid_labels = ['neutral', 'contradiction', 'entailment']
    type_data = type_data[type_data['gold_label'].isin(valid_labels)].copy()
    type_data['label1'] = type_data['gold_label']
    return type_data
snli_train = read('train')
snli_dev = read('dev')
snli_train.head()

,gold_label,sentence1,sentence2,pairID,label1
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4r1n,neutral
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4r1c,contradiction
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4r1e,entailment
3,neutral,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2r1n,neutral
4,entailment,Children smiling and waving at camera,There are children present,2267923837.jpg#2r1e,entailment


In [ ]:
snli_train['index'] = range(len(snli_train))
type = 'SNLI_train'
df = snli_train[['index', 'pairID', 'sentence1', 'sentence2', 'gold_label']]
df['type'] = type
df.rename(columns={'gold_label': 'label'}, inplace=True)
df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]

pathName = 'SNLI'
biased_path = f'./gen_{pathName}/'
if not os.path.exists(biased_path):
    os.makedirs(biased_path)
df.to_json(os.path.join(biased_path, f'{type}.jsonl'), orient='records', lines=True)
df.head()

,index,pairID,sentence1,sentence2,label,type
0,0,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,neutral,SNLI_train
1,1,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",contradiction,SNLI_train
2,2,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",entailment,SNLI_train
3,3,2267923837.jpg#2r1n,Children smiling and waving at camera,They are smiling at their parents,neutral,SNLI_train
4,4,2267923837.jpg#2r1e,Children smiling and waving at camera,There are children present,entailment,SNLI_train


In [103]:
devdev = pd.DataFrame(columns=['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type'])
devdev

,index,pairID,sentence1,sentence2,label,type


In [104]:
task = 'QNLI'
def qnli_read(type='dev'):
    df = pd.read_csv(f"./raw/QNLI/{type}.tsv", sep='\t', encoding='utf-8', on_bad_lines='skip').rename(columns={"question": "sentence1", "sentence": "sentence2"})
    df['pairID'] = range(len(df))
    df['type'] = f'QNLI_{type}'
    df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
    df.to_json(os.path.join(biased_path, f'QNLI_{type}.jsonl'), orient='records', lines=True)
    return df
QNLI_dev = qnli_read(type='dev')
QNLI_dev.head()
devdev = pd.concat([devdev, QNLI_dev], axis=0)
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
5261,5458,5261,How many people lived in Warsaw in 1939?,Unfortunately this belief still lives on in Po...,not_entailment,QNLI_dev
5262,5459,5262,One of FIS' agenda items was to force women to...,Taking advantage of economic failure and unpop...,entailment,QNLI_dev
5263,5460,5263,Dendritic cells are named that because they re...,Dendritic cells serve as a link between the bo...,not_entailment,QNLI_dev
5264,5461,5264,When did it retract to a inland formation?,"However, the rainforest still managed to thriv...",not_entailment,QNLI_dev


In [105]:
def mnli_read(type='dev_matched'):
    df = pd.read_csv(f"./raw/MNLI/{type}.tsv", sep='\t', encoding='utf-8', on_bad_lines='skip').rename(columns={"question": "sentence1", "sentence": "sentence2"})\
        [['index', 'pairID', 'sentence1', 'sentence2', 'gold_label']]
    df.rename(columns={'gold_label': 'label'}, inplace=True)
    df['type'] = f"MNLI_{type}"
    df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
    df.to_json(os.path.join(biased_path, f'MNLI_{type}.jsonl'), orient='records', lines=True)
    return df
mnli_dev_matched = mnli_read(type='dev_matched')
mnli_dev_mismatched = mnli_read(type='dev_mismatched')
mnli_dev_mismatched.head()
devdev = pd.concat([devdev, mnli_dev_matched, mnli_dev_mismatched], axis=0)
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
9827,9827,47686c,Do you watch that?,Can you see?,contradiction,MNLI_dev_mismatched
9828,9828,126829c,"To a Western ear, the most predictable of lang...","To the Western ear, the least predictable of l...",contradiction,MNLI_dev_mismatched
9829,9829,37368c,The recorder captured the sounds of loud thump...,The recorder didn't capture any of the sounds.,contradiction,MNLI_dev_mismatched
9830,9830,97460n,That's a good attitude!,"You feel good about this, don't you?",neutral,MNLI_dev_mismatched


In [106]:
df = pd.read_json(f"./raw/HANS/test.jsonl", lines=True, encoding='utf-8')
df.rename(columns={'gold': 'label', 'premise':'sentence1', 'hypothesis': 'sentence2'}, inplace=True)
df['type'] = 'HANS_' + df['heuristic']
df['index'] = range(len(df))
df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
df.to_json(os.path.join(biased_path, f'HANS_test.jsonl'), orient='records', lines=True)
df.head()
devdev = pd.concat([devdev, df], axis=0)
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
29995,29995,ex29995,Obviously the senator avoided the professor .,The senator avoided the professor .,entailment,HANS_constituent
29996,29996,ex29996,Certainly the senators recognized the actor .,The senators recognized the actor .,entailment,HANS_constituent
29997,29997,ex29997,Obviously the artists ran .,The artists ran .,entailment,HANS_constituent
29998,29998,ex29998,Suddenly the tourists supported the judges .,The tourists supported the judges .,entailment,HANS_constituent


In [107]:
df = pd.read_json(f"./raw/MNLI_hard_m/test.jsonl", lines=True, encoding='utf-8')
df.rename(columns={'gold_label': 'label'}, inplace=True)
df['type'] = 'MNLI_hard_m'
df['index'] = range(len(df))
df['pairID'] = range(len(df))
df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
df.to_json(os.path.join(biased_path, f'MNLI_hard_m.jsonl'), orient='records', lines=True)
devdev = pd.concat([devdev, df], axis=0)
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
4568,4568,4568,"He went down on his knees , examining it minut...",It smelled like eggs .,neutral,MNLI_hard_m
4569,4569,4569,"Under Ferdinand and Isabella , Spain underwent...",Ferdinand and Isabella caused stunning changes...,entailment,MNLI_hard_m
4570,4570,4570,7 ) Nonautomated First-Class and Standard-A ma...,Nonautomated First-Class and Standard-A mailer...,entailment,MNLI_hard_m
4571,4571,4571,"Finally , the FDA will conduct workshops , iss...",The FDA is set to conduct workshops .,entailment,MNLI_hard_m


In [108]:
df = pd.read_json(f"./raw/MNLI_hard_mm/test.jsonl", lines=True, encoding='utf-8')
df.rename(columns={'gold_label': 'label'}, inplace=True)
df['type'] = 'MNLI_hard_mm'
df['index'] = range(len(df))
df['pairID'] = range(len(df))
df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
df.to_json(os.path.join(biased_path, f'MNLI_hard_mm.jsonl'), orient='records', lines=True)
devdev = pd.concat([devdev, df], axis=0)
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
4525,4525,4525,"For Indianapolis , that public university must...",IUPUI is in the city of Chicago .,contradiction,MNLI_hard_mm
4526,4526,4526,Please donate today .,Our website is down please come back tomorrow ...,contradiction,MNLI_hard_mm
4527,4527,4527,Do you watch that ?,Can you see ?,contradiction,MNLI_hard_mm
4528,4528,4528,That 's a good attitude !,"You feel good about this , do n't you ?",neutral,MNLI_hard_mm


In [109]:
df = pd.read_json(f"./raw/ST/test.jsonl", lines=True, encoding='utf-8')[['pairID', 'sentence1', 'sentence2', 'gold_label']]
df.rename(columns={'gold_label': 'label'}, inplace=True)
df['type'] = 'ST'
df['index'] = range(len(df))
df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
df = df.sample(n=30000, random_state=3407)
df.to_json(os.path.join(biased_path, f'ST.jsonl'), orient='records', lines=True)
devdev = pd.concat([devdev, df], axis=0)
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
31279,31279,15447c,'You should do the fixing.',"""I'll take care of fixing this and false is no...",contradiction,ST
45159,45159,102826c,we were lucky in that in one respect in that a...,She still had comolete awareness after her str...,contradiction,ST
3705,3705,169013e,Do you know what this is and true is true and ...,She wore a coat and carried a small badge.,entailment,ST
14557,14557,176663e,The whole thing and true is true and true is t...,"In your childhood, reading what books was plea...",contradiction,ST


In [110]:
df = pd.read_json(f"./raw/IS_CS/snli_with_misleading_score/snli_with_misleading_rate.jsonl", lines=True, encoding='utf-8')[['pairID', 's1', 's2', 'gold_label']]
df.rename(columns={'gold_label': 'label', 's1': "sentence1", 's2': "sentence2"}, inplace=True)
df['type'] = 'IS_CS_snli'
df['index'] = range(len(df))
df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
df.to_json(os.path.join(biased_path, f'IS_CS_snli.jsonl'), orient='records', lines=True)
df.head()
devdev = pd.concat([devdev, df], axis=0)
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
9837,9837,77063034.jpg#3r1e,A small girl wearing a pink jacket is riding o...,The carousel is moving.,entailment,IS_CS_snli
9838,9838,77063034.jpg#3r1c,A small girl wearing a pink jacket is riding o...,The girl is moving at the speed of light.,contradiction,IS_CS_snli
9839,9839,4805835848.jpg#0r1e,A young girl with blue and pink ribbons in her...,People in a water fountain,entailment,IS_CS_snli
9840,9840,4805835848.jpg#0r1c,A young girl with blue and pink ribbons in her...,A young girl knits a sweater,contradiction,IS_CS_snli


In [9]:
import pandas as pd
df = pd.read_csv(f"./raw/IS_CS/ADDAMOD/add_amod(dev).tsv", sep='\t', encoding='utf-8', on_bad_lines='skip',header=None)
df.columns = ['sentence1', 'sentence2', 'label']
df['label'].unique()

array(['neutral'], dtype=object)

In [ ]:
df['type'] = 'IS_CS_ADDAMOD'
df['index'] = range(len(df))
df['pairID'] = range(len(df))
df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
df.to_json(os.path.join(biased_path, f'IS_CS_ADDAMOD.jsonl'), orient='records', lines=True)
df.head()
devdev = pd.concat([devdev, df], axis=0)
devdev

In [112]:
df = pd.read_csv(f"./raw/IS_CS/SUBOBJSWAP/sub_obj_swap(dev).tsv", sep='\t', encoding='utf-8', on_bad_lines='skip',header=None)
df.columns = ['sentence1', 'sentence2', 'label']
df['type'] = 'IS_CS_SUBOBJSWAP'
df['index'] = range(len(df))
df['pairID'] = range(len(df))
df = df[['index', 'pairID', 'sentence1', 'sentence2', 'label', 'type']]
df.to_json(os.path.join(biased_path, f'IS_CS_SUBOBJSWAP.jsonl'), orient='records', lines=True)
df.head()
devdev = pd.concat([devdev, df], axis=0)
devdev


,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
966,966,966,A view from a hidden camera at a low height sh...,A man from a hidden camera at a low height sho...,entailment,IS_CS_SUBOBJSWAP
967,967,967,A policeman is directing a crowd of people wal...,A crowd is directing a policeman of people wal...,entailment,IS_CS_SUBOBJSWAP
968,968,968,Three young indian children play wiffle ball .,Three young indian ball play wiffle children .,entailment,IS_CS_SUBOBJSWAP
969,969,969,A man is performing an aerial skateboard trick...,A trick is performing an aerial skateboard man...,entailment,IS_CS_SUBOBJSWAP


In [113]:
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
966,966,966,A view from a hidden camera at a low height sh...,A man from a hidden camera at a low height sho...,entailment,IS_CS_SUBOBJSWAP
967,967,967,A policeman is directing a crowd of people wal...,A crowd is directing a policeman of people wal...,entailment,IS_CS_SUBOBJSWAP
968,968,968,Three young indian children play wiffle ball .,Three young indian ball play wiffle children .,entailment,IS_CS_SUBOBJSWAP
969,969,969,A man is performing an aerial skateboard trick...,A trick is performing an aerial skateboard man...,entailment,IS_CS_SUBOBJSWAP


In [114]:
devdev['label'] = devdev['label'].replace({
    'non-entailment': 'not_entailment'
})
devdev['label'].value_counts()

label
entailment        42345
neutral           23460
contradiction     23040
not_entailment    17666
Name: count, dtype: int64

In [115]:
devdev['type'].value_counts()

type
ST                      30000
HANS_lexical_overlap    10000
HANS_subsequence        10000
HANS_constituent        10000
IS_CS_snli               9842
MNLI_dev_mismatched      9832
MNLI_dev_matched         9714
QNLI_dev                 5266
MNLI_hard_m              4573
MNLI_hard_mm             4530
IS_CS_ADDAMOD            1783
IS_CS_SUBOBJSWAP          971
Name: count, dtype: int64

In [9]:
pathName = 'SNLI'
biased_path = f'./gen_{pathName}/'
devdev.to_json(os.path.join(biased_path, f'SNLI_gen_dev.jsonl'), orient='records', lines=True)

In [10]:
devdev = pd.read_json(f"./gen_SNLI/SNLI_gen_dev.jsonl", lines=True, encoding='utf-8')
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
106506,106506,106506,A view from a hidden camera at a low height sh...,A man from a hidden camera at a low height sho...,entailment,IS_CS_SUBOBJSWAP
106507,106507,106507,A policeman is directing a crowd of people wal...,A crowd is directing a policeman of people wal...,entailment,IS_CS_SUBOBJSWAP
106508,106508,106508,Three young indian children play wiffle ball .,Three young indian ball play wiffle children .,entailment,IS_CS_SUBOBJSWAP
106509,106509,106509,A man is performing an aerial skateboard trick...,A trick is performing an aerial skateboard man...,entailment,IS_CS_SUBOBJSWAP


In [6]:
devdev['index'] = range(len(devdev))
devdev['pairID'] = devdev['index'].astype(str)
devdev

,index,pairID,sentence1,sentence2,label,type
0,0,0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,QNLI_dev
1,1,1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment,QNLI_dev
2,2,2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment,QNLI_dev
3,3,3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,QNLI_dev
4,4,4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment,QNLI_dev
...,...,...,...,...,...,...
106506,106506,106506,A view from a hidden camera at a low height sh...,A man from a hidden camera at a low height sho...,entailment,IS_CS_SUBOBJSWAP
106507,106507,106507,A policeman is directing a crowd of people wal...,A crowd is directing a policeman of people wal...,entailment,IS_CS_SUBOBJSWAP
106508,106508,106508,Three young indian children play wiffle ball .,Three young indian ball play wiffle children .,entailment,IS_CS_SUBOBJSWAP
106509,106509,106509,A man is performing an aerial skateboard trick...,A trick is performing an aerial skateboard man...,entailment,IS_CS_SUBOBJSWAP


In [9]:
devdev[devdev['label'] == 'not_entailment']['type'].value_counts()

type
HANS_lexical_overlap    5000
HANS_subsequence        5000
HANS_constituent        5000
QNLI_dev                2666
Name: count, dtype: int64